## Accuracy

In [1]:
# Definisci il nuovo prompt
PROMPT = """Compare the gold answer with the student's final answer to determine if they are equivalent. Respond exclusively with 'yes' or 'no' without providing any additional explanation. If the student's final answer is empty, respond with 'no'.

Use this exact output format: "Answer: yes/no"

Gold answer:
{}

Final answer:
{}
"""

In [2]:
from google.colab import userdata
from huggingface_hub import login
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)

ModuleNotFoundError: No module named 'google.colab'

In [3]:
from dotenv import load_dotenv
from huggingface_hub import login
import os

load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
login(token=HF_TOKEN)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
from datasets import load_dataset
dataset = load_dataset("lozziopadredeivizzi/mathematic_games_dataset_en", split="train")
#dataset = load_dataset("alecocc/mathematic_games_dataset_en_2024_def", split="train")


In [22]:
import json
import pandas as pd

data = []
with open('../out/fileJSONL/Qwen2.5-72B/pass1/cot/completions_qwen2.5-72B_cot_pass1.jsonl', 'r') as file:
    for line in file:
        data.append(json.loads(line))

In [23]:
def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [24]:
results = []

for item in data:
    # Estrai il campo 'final_answer'
    id = item['id']
    final_answer = item['final_answer']
    gold_answer = item['gold_answer']

    if is_float(final_answer):
      if final_answer in gold_answer:
          answer = 'yes'
          results.append({
              "id":id,
              "gold_answer": gold_answer,
              "final_answer": final_answer,
              "model_response": answer
          })
      else:
          answer = 'no'
          results.append({
              "id":id,
              "gold_answer": gold_answer,
              "final_answer": final_answer,
              "model_response": answer
          })
    elif final_answer == "":
      answer = 'no'
      results.append({
          "id":id,
          "gold_answer": gold_answer,
          "final_answer": final_answer,
          "model_response": answer
      })
    else:
      answer = 'llm'
      results.append({
          "id":id,
          "gold_answer": gold_answer,
          "final_answer": final_answer,
          "model_response": answer
      })

# Crea un DataFrame dai risultati
df_result = pd.DataFrame(results)

In [25]:
df_result

,id,gold_answer,final_answer,model_response
0,2,Amerigo will be 58 years old,58,yes
1,6,The 2017th issue is 0,0,yes
2,7,Carla used 48 tiles,48,yes
3,8,Nando takes 30 minutes,30,yes
4,9,Liliana lost 49 Euro,2179,no
...,...,...,...,...
1402,2175,464 mm,208,no
1403,2176,19,0,no
1404,2177,6 members,13,no
1405,2178,75 cents,75,yes


In [26]:
df_result[df_result['model_response'] == 'yes']

,id,gold_answer,final_answer,model_response
0,2,Amerigo will be 58 years old,58,yes
1,6,The 2017th issue is 0,0,yes
2,7,Carla used 48 tiles,48,yes
3,8,Nando takes 30 minutes,30,yes
5,11,The ratio is worth 25 times the product,25,yes
...,...,...,...,...
1393,2147,1,1,yes
1394,2149,20 apples,20,yes
1399,2168,10 cards,10,yes
1400,2172,10 lots,10,yes


In [27]:
ids_to_modify = [29,435,592,895,1062,1332,1401,1562,1597,1780,1880,1942]

In [28]:
# Iterate through the DataFrame and modify the 'model_response' for specified IDs
for index, row in df_result.iterrows():
  if row['id'] in ids_to_modify:
    df_result.loc[index, 'model_response'] = 'no'

In [29]:
df_result_2 = df_result[df_result['model_response'] == 'llm']
df_result_2

,id,gold_answer,final_answer,model_response
32,25,The first term a1 is 4 and the reason d is 6,"a_1 = 1, d = 0",llm
45,41,The helicopter will be located 10 m south and ...,20 \text{ meters West and 10 meters South},llm
48,44,The helicopter will be located 10m south and 2...,10\sqrt{5},llm
52,51,The result of the multiplication is 6952 or 78...,2 \times 69 = 138,llm
59,60,At: 7 hours; 18 minutes; 45 seconds.,7:00 \text{ AM},llm
...,...,...,...,...
1365,2101,27/3,22 \text{ March},llm
1370,2111,63000,963000 \text{ m}^2,llm
1371,2112,E,E,llm
1375,2117,1000 (1×0=0+0),1101 \text{ and } 9299,llm


# OpenAI evaluation

In [30]:
from openai import OpenAI

client = OpenAI(
    api_key=""
)

In [31]:
def make_completion(gold, final_answer):
    system_prompt = """Compare the gold answer with the student's final answer to determine if they are equivalent. Respond exclusively with 'yes' or 'no' without providing any additional explanation.

Use this exact output format: "Answer: yes/no"""
    try:
        # Create a chat completion using the question and context
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages = [
                {"role": "system","content": system_prompt},
                {"role": "user", "content": f"Gold answer: {gold}\nFinal Answer: {final_answer}"},
            ],
            temperature=0,
            max_tokens=100,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            seed=42
        )

        return response
    except Exception as e:
        print(e)
        return ""

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
from collections import Counter

NUM_ITEMS = len(df_result_2)
# Ciclo su ciascuna istanza nel file JSON, usando tqdm per la barra di avanzamento
k = -1
evaluations = []
for _, row in tqdm(df_result_2.iterrows(), desc="Processing items", total=NUM_ITEMS):
    k += 1
    if k < NUM_ITEMS:
        # Estrai i campi 'gold_answer' e 'final_answer'
        gold_answer = row['gold_answer']
        final_answer = row['final_answer']

        # Crea il prompt con i campi estratti
        response = make_completion(gold=gold_answer, final_answer=final_answer)
        completion = response.choices[0].message.content.strip()
        model = response.model
        #usage = dict(response.usage)

        evaluations.append(completion.lower().replace("answer:", "").strip())

        with open(f'../out/eval.jsonl', 'a') as f:
            result = {
                "model": model,
                "id": row['id'],
                "decision": completion,
                "gold_answer": gold_answer,
                "final_answer": final_answer,
                #"usage": usage if completion else {},
            }
            json.dump(result, f, ensure_ascii=False)
            f.write('\n')
    else:
        break


Processing items:  25%|██████████████████████████▏                                                                             | 84/333 [00:56<03:10,  1.31it/s]

In [ ]:
assert len(evaluations) == len(df_result_2)
df_result_2['model_response'] = evaluations

In [ ]:
df_result_2

In [ ]:
df_final = pd.concat([df_result_2, df_result[df_result['model_response'] != 'llm']], ignore_index=True)

In [ ]:
df_final.to_csv('../out/fileJSONL/Qwen2.5-72B/pass1/cot/qwen2_5_72b_pass1_cot.csv', index=False)

In [ ]:
yes_count = df_final['model_response'].value_counts().get('yes', 0)
print(f"The value 'yes' appears {yes_count} times in the 'model_response' column.")

In [ ]:
# Calcola l'accuracy
accuracy = yes_count /df_final.shape[0]
print(f"Accuracy: {accuracy:.2%}")